In [2]:
import sqlite3 as sq
import pandas as pd
import numpy as np
from pandas import DataFrame as df
# from interval import Interval as intt

In [233]:
# 分数预估
conn = sq.connect('高考数据库.sqlite3')     # 连接数据库
c = conn.cursor()

c.execute("SELECT name FROM sqlite_master WHERE type='table'")     # 读取表名
table = c.fetchall()
table
cates = table[10][0]     # 获取类名
mmajors = table[14][0]     # 获取专业表名
colleges = table[15][0]     # 获取学校名
province = table[12][0]
rankings = table[13][0]
fir = table[11][0]

# 获取表的列名、列名结构
def col(form):
    c.execute("SELECT * FROM {}".format(form))
    col_name = [tuple[0] for tuple in c.description]
    
    c.execute("PRAGMA table_info({})".format(form))
    col_structure = c.fetchall()
    
    return col_name,col_structure

# 获取表的内容（DF结构）
def f_df(form):
    sql = "SELECT * FROM {}".format(form)
    values = c.execute(sql)
    f_df = pd.DataFrame(data = values)
    return f_df

In [3]:
# 河南一分一档表
rankings21 = f_df(rankings)
rankings21.columns = col(rankings)[0]
rankings21 = rankings21[rankings21['year'] == 2021]
rankings21 = rankings21[rankings21['provinceID_id'] == 1]
colleges21 = f_df(colleges)

NameError: name 'f_df' is not defined

In [15]:
rankings21=pd.read_csv('7.21/ranking.csv',encoding='gbk',names=['iid','year','score','rank','categoryID_id','provinceID_id'])
rankings21 = rankings21[rankings21['year'] == 2021]
rankings21 = rankings21[rankings21['provinceID_id'] == 1]
rankings21

,iid,year,score,rank,categoryID_id,provinceID_id
31558,2777554,2021,675,10,1,1
31559,2779899,2021,674,15,1,1
31560,2782245,2021,673,16,1,1
31561,2784592,2021,672,22,1,1
31562,2786940,2021,671,26,1,1
...,...,...,...,...,...,...
32738,6240264,2021,104,466448,2,1
32739,6243789,2021,103,466460,2,1
32740,6247315,2021,102,466470,2,1
32741,6250842,2021,101,466478,2,1


In [21]:
colleges21=pd.read_csv('7.21/college.csv',encoding='gbk',header=None)
colleges21

,0,1,2,3,4,5
0,10001,北京大学,1,1,1,25
1,10002,中国人民大学,1,1,1,25
2,10003,清华大学,1,1,1,25
3,10004,北京交通大学,1,1,0,25
4,10006,北京航空航天大学,1,1,1,25
...,...,...,...,...,...,...
98,11413,中国矿业大学（北京）,0,1,1,25
99,11415,中国地质大学（北京）,0,1,1,25
100,11417,北京联合大学,0,0,0,25
101,20000,复旦大学医学部,1,1,1,9


In [78]:
# 专业学校推荐
alldf = pd.read_csv('alldata1.0.csv',encoding='gbk',index_col=0)

def rec(cate,score,prov,c_rank,major,m_rank,risk):
    df = alldf[alldf['categoryID_id'] == cate]     # 筛选文理科
    
    rec_rank = rankings21[rankings21['categoryID_id'] == cate]     # 河南2021一分一档对照表
    ranking = rec_rank[rec_rank['score'] == score]['rank'].values[0]
    print(ranking)
    
    df = df[df['c1'] == major[0]]
    if major[1] != None:
        df = df[df['c2'] == major[1]]
#     df = df[df['provinceLoc'] == prov]     # 筛选城市
    
    # 统计综合得分：城市重要度和专业重要度范围为1-10
    
    df['sum_score'] = (df['cityScore']*c_rank + df['majorScore']*m_rank) / (c_rank + m_rank)
    # 学校代码转换学校名称
    for dfs in df['collegeID_id']:
        df.loc[df['collegeID_id'] == dfs,'college'] = colleges21[colleges21[0] == dfs][1].values[0]
    df = df.sort_values(['preRanking'],ascending=True)
#     print(df)
    # 报考排名范围：0.9ranking-1.5ranking
    if risk == 1:     # 输出所有符合条件的推荐方案
        df = df[df['preRanking'] <= ranking*1.5]    
        df = df[df['preRanking'] >= ranking*0.9]
        if df.empty:
            df = df[df['preRanking'] >= ranking*1.0][0:6]
    elif risk ==2:     # 输出报考风险相对大的推荐方案
#         df = df[df['preRanking'] < ranking*1.0]    
        df = df[df['preRanking'] >= ranking*0.9]
        df = df[0:6]
    else:     # 输出报考稳妥的推荐方案 
#         df = df[df['preRanking'] < ranking*1.5]    
        df = df[df['preRanking'] >= ranking*1.0]
        df = df[0:6]
    df = df.sort_values(['sum_score'],ascending=False)
    df = df.loc[:,('provinceLoc','cityLoc','college','majorName','preRanking')]
    
    return df
   

In [152]:
# 专业学校推荐
alldf = pd.read_csv('alldata1.0.csv',encoding='gbk',index_col=0)
alldf['c2']=alldf['c2'].fillna('')

def rec(cate,score,prov,c_rank,majors,m_rank,risk):
    '''
    输入为：文理科，高考分数，期望省份，地域重要性，期望专业（多选），专业重要性，风险性
    输出为：该推荐大学+专业的省市位置，大学名，专业名，预计所需最低排名，推荐指数
    '''
    
    # 筛选文理科
    df = alldf[alldf['categoryID_id'] == cate]     
    
    # 河南2021一分一档对照表
    rec_rank = rankings21[rankings21['categoryID_id'] == cate]     
    if score>=rec_rank.describe()['score']['max']:
        ranking=rec_rank.describe()['rank']['min']
    else:
        ranking = rec_rank[rec_rank['score'] == score]['rank'].values[0]
    print(ranking)
    
    df_pass=pd.DataFrame()
    
    for major in majors:
        # 筛选专业
        if major[1]==None:#输入 c1 None
            df_temp = df[df['c1'].str.contains(major[0])]
            df_pass = pd.concat([df_pass,df_temp],axis=0 ,ignore_index=True)
        else:#输入c1 c2
            df_temp = df[df['c2'].str.contains(major[1])]
            df_temp1 = df[(df['c1'].str.contains(major[0])) & (df['c2']=='')]
            df_pass = pd.concat([df_pass,df_temp,df_temp1],axis=0 ,ignore_index=True)
        
    df=df_pass.drop_duplicates()
    
    # 统计综合得分
    for provID in df['provinceLoc']:
        if provID == prov:
            df.loc[df['provinceLoc'] == prov,'sum_score'] = (df['cityScore']*(1 + c_rank/10) + df['majorScore']*m_rank) / (c_rank + m_rank)
        else:
            df.loc[df['provinceLoc'] != prov,'sum_score'] = (df['cityScore']*1 + df['majorScore']*m_rank) / (c_rank + m_rank)
            
    
    # 学校代码转换学校名称
    for dfs in df['collegeID_id']:
        df.loc[df['collegeID_id'] == dfs,'college'] = colleges21[colleges21[0] == dfs][1].values[0]
    df = df.sort_values(['preRanking'],ascending=True)

    # 报考排名范围：0.9ranking-1.5ranking
    if risk == 1:     # 输出所有符合条件的推荐方案
        df_temp1 = df[df['preRanking'] >= ranking*0.9]
        df_temp2 = df_temp1[df_temp1['preRanking'] < ranking*1.0]
        df_temp3 = df[df['preRanking'] >= ranking*1.0][0:6]
        df = pd.concat([df_temp2,df_temp3],axis=0 ,ignore_index=True)
        df = df[0:12]
        if df.empty:
            df = df[df['preRanking'] >= ranking*1.0][0:6]
    elif risk ==2:     # 输出报考风险相对大的推荐方案
#         df = df[df['preRanking'] < ranking*1.0]    
        df = df[df['preRanking'] >= ranking*0.9]
        df = df[0:6]
    else:     # 输出报考稳妥的推荐方案 
#         df = df[df['preRanking'] < ranking*1.5]    
        df = df[df['preRanking'] >= ranking*1.0]
        df = df[0:6]
    
    max_score=df.describe()['sum_score']['max']
    df['sum_score']=df['sum_score']/max_score*100
        
    df = df.sort_values(['sum_score'],ascending=False)
    df = df.loc[:,('provinceLoc','cityLoc','college','majorName','preRanking','sum_score')]
    
    return df
   

In [136]:
rec_rank = rankings21[rankings21['categoryID_id'] == 1]
rec_rank.describe()['score']['max']

675.0

In [153]:
rec(1,672,'北京市',1,[['文学',None]],1,1)

22


E:\anaconda\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
E:\anaconda\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
E:\anaconda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,provinceLoc,cityLoc,college,majorName,preRanking,sum_score
0,北京市,北京市,北京大学,中国语言文学类,33,100.000000
5,北京市,北京市,中国人民大学,新闻传播学类,92,100.000000
1,北京市,北京市,北京大学,新闻传播学类,33,95.004795
3,上海市,上海市,复旦大学,中国语言文学类,46,91.567096
4,上海市,上海市,复旦大学,新闻传播学类,75,91.567096
2,福建省,厦门市,厦门大学,外国语言文学类（外国语言文学加会计或财务）,46,70.167639


In [122]:
for i in range(500,710,10):
    print(i)
    print(rec(2,i,'江苏省',10,[['工学','计算机类']],30,1)[0:10])

500
142881


E:\anaconda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Empty DataFrame
Columns: [provinceLoc, cityLoc, college, majorName, c2, preRanking, sum_score]
Index: []
510
128279


E:\anaconda\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
E:\anaconda\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
E:\anaconda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

Empty DataFrame
Columns: [provinceLoc, cityLoc, college, majorName, c2, preRanking, sum_score]
Index: []
520
113880


...

  provinceLoc cityLoc   college             majorName    c2  preRanking  \
1         江苏省     南京市      南京大学                  软件工程  计算机类         627   
3         湖北省     武汉市    华中科技大学    计算机科学与技术(启明本硕博实验班)  计算机类         953   
4         湖北省     武汉市    华中科技大学  计算机科学与技术(计算机卓越计划实验班)  计算机类         953   
0         安徽省     合肥市  中国科学技术大学                  计算机类  计算机类         627   
2         北京市     北京市  北京航空航天大学      工科试验班类(国际通用工程学院)               953   
5        黑龙江省    哈尔滨市   哈尔滨工业大学        工科试验班（英才本硕博连读）              1159   
6         北京市     北京市    北京理工大学                徐特立英才班              1377   
7         湖北省     武汉市      武汉大学                 工科试验班              1521   

   sum_score  
1  73.595500  
3  71.513750  
4  71.513750  
0  69.020250  
2  68.985647  
5  68.046955  
6  67.063557  
7  63.751250  
690
255


E:\anaconda\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
E:\anaconda\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
E:\anaconda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

  provinceLoc cityLoc   college        majorName    c2  preRanking  sum_score
4         江苏省     南京市      南京大学         计算机科学与技术  计算机类         556  73.595500
2         浙江省     杭州市      浙江大学  工科试验班（竺可桢学院图灵班）               298  69.281750
5         安徽省     合肥市  中国科学技术大学             计算机类  计算机类         627  69.020250
3         安徽省     合肥市  中国科学技术大学     工科试验班（科技英才班）               491  67.145250
0         上海市     上海市      复旦大学             软件工程  计算机类         268  66.757750
1         上海市     上海市      复旦大学            工科试验班               298  64.924417
700
68


E:\anaconda\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
E:\anaconda\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
E:\anaconda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

  provinceLoc cityLoc   college        majorName    c2  preRanking  sum_score
0         北京市     北京市      清华大学             计算机类  计算机类          65  74.368000
1         北京市     北京市      北京大学             计算机类  计算机类          79  74.368000
2         北京市     北京市      北京大学           工科试验班类               107  70.284667
5         浙江省     杭州市      浙江大学  工科试验班（竺可桢学院图灵班）               298  69.281750
6         安徽省     合肥市  中国科学技术大学     工科试验班（科技英才班）               491  67.145250
3         上海市     上海市      复旦大学             软件工程  计算机类         268  66.757750
4         上海市     上海市      复旦大学            工科试验班               298  64.924417


In [82]:
alldf[alldf['c2'].str.contains('计算机类')]

,majorName,year,minScore,categoryID_id,collegeID_id,c1,c2,college_loc,provinceLoc,cityLoc,adcode,cityScore,c2code,majorScore,preRanking
id,,,,,,,,,,,,,,,
0,计算机类,2020,622,2.0,10593,工学,计算机类,广西省南宁市,广西壮族自治区,南宁市,450100,41.02,['0812'],80.000000,25751
17,计算机科学与技术,2020,647,2.0,10559,工学,计算机类,广东省广州市,广东省,广州市,440100,80.36,['0812'],83.000000,10957
22,物联网工程,2020,643,2.0,10559,工学,计算机类,广东省广州市,广东省,广州市,440100,80.36,['0812'],83.000000,12897
23,网络空间安全,2020,643,2.0,10559,工学,计算机类,广东省广州市,广东省,广州市,440100,80.36,['0812'],83.000000,12897
185013,数据科学与大数据技术,2020,655,2.0,10036,工学,计算机类,北京市,北京市,北京市,110000,94.72,['0812'],87.653846,7724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5141020,数据科学与大数据技术,2020,615,2.0,10560,工学,计算机类,广东省汕头市,广东省,汕头市,440500,22.00,['0812'],73.846154,31039
5197765,计算机类,2020,616,2.0,10224,工学,计算机类,黑龙江省哈尔滨市,黑龙江省,哈尔滨市,230100,45.06,['0812'],79.297619,30260
5302639,计算机类,2020,619,2.0,10225,工学,计算机类,黑龙江省哈尔滨市,黑龙江省,哈尔滨市,230100,45.06,['0812'],81.000000,27959


In [146]:
colleges21[colleges21[1]=='中国人民大学']

,0,1,2,3,4,5
1,10002,中国人民大学,1,1,1,25


In [144]:
alldf[alldf['collegeID_id']==10002]

,majorName,year,minScore,categoryID_id,collegeID_id,c1,c2,college_loc,provinceLoc,cityLoc,adcode,cityScore,c2code,majorScore,preRanking
id,,,,,,,,,,,,,,,
1610584,财政学类,2020,691,2.0,10002,经济学,财政学类,北京市,北京市,北京市,110000,94.72,['0202'],96.000000,556
1612264,法学,2020,695,2.0,10002,法学,法学类,北京市,北京市,北京市,110000,94.72,['0301'],96.000000,335
1613945,工商管理类,2020,689,2.0,10002,管理学,工商管理类,北京市,北京市,北京市,110000,94.72,['1202'],96.000000,705
1615627,国际政治,2020,687,2.0,10002,法学,政治学类,北京市,北京市,北京市,110000,94.72,['0302'],93.000000,848
1617310,金融学类,2020,695,2.0,10002,经济学,金融学类,北京市,北京市,北京市,110000,94.72,['0202'],96.000000,335
1618994,经济学类,2020,692,2.0,10002,经济学,经济学类,北京市,北京市,北京市,110000,94.72,['0201'],96.000000,491
1620679,理科试验班,2020,689,2.0,10002,理学,,北京市,北京市,北京市,110000,94.72,['0700'],82.200000,705
1622365,人力资源管理,2020,688,2.0,10002,管理学,工商管理类,北京市,北京市,北京市,110000,94.72,['1202'],96.000000,774
1624052,社会科学试验班（管理学科类）,2020,687,2.0,10002,管理学,,北京市,北京市,北京市,110000,94.72,['1200'],88.800000,848


## 用不上的函数

In [102]:
# 暂时用不上的函数
def mins(college,major):
    df = alldf[alldf['collegeID_id'] == college]
    df = df[df['c1'] == major[0]]
    df = df[df['c2'] == major[1]]
    minscore = df[df['minScore'] == min(df['minScore'])]
    return minscore

True

In [269]:
ttt = rankings21[rankings21['score'] == 750]
ttt.empty

True

In [273]:
rankings21.shape[0]

1185